In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("myApp").enableHiveSupport().config("hive", "true").getOrCreate()

23/02/22 16:01:12 WARN Utils: Your hostname, TIGER03531 resolves to a loopback address: 127.0.1.1; using 172.20.148.2 instead (on interface eth0)
23/02/22 16:01:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 16:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("people.json")

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.columns

['age', 'name']

In [7]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [8]:
from pyspark.sql import Row

In [9]:
df["age"]

Column<'age'>

In [10]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [11]:
df.head(2)[0]

Row(age=None, name='Michael')

In [12]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [13]:
df.withColumn("newage", df["age"] * 2).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    60|
|  19| Justin|    38|
+----+-------+------+



In [14]:
df.withColumnRenamed("age","my new age").show()

+----------+-------+
|my new age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [15]:
df.createOrReplaceTempView("people")

In [16]:
results = spark.sql("SELECT * FROM people")

In [17]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [18]:
age = 100
temp = 'le'
new_results = spark.sql(f"""SELECT * FROM peop{temp}
                        WHERE age > 0 and age < {age}
                        LIMIT 100""")

In [19]:
new_results.show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+



In [22]:
df2 = spark.read.csv("appl_stock.csv", inferSchema= True, header = True)

In [23]:
df2.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [24]:
df2.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [25]:
df2.filter((df2["Close"] < 500) & (df2["Open"] > 300)).select(['Date',"Close","Volume"]).show()

+-------------------+------------------+---------+
|               Date|             Close|   Volume|
+-------------------+------------------+---------+
|2010-10-13 00:00:00|300.14000699999997|157523100|
|2010-10-14 00:00:00|302.30998999999997|108824100|
|2010-10-15 00:00:00|314.73999399999997|230548500|
|2010-10-18 00:00:00|        317.999989|273252700|
|2010-10-19 00:00:00|309.48999399999997|308196000|
|2010-10-20 00:00:00|        310.529991|180406100|
|2010-10-21 00:00:00|        309.520008|137865000|
|2010-10-22 00:00:00|307.47000099999997| 93194500|
|2010-10-25 00:00:00|        308.839993| 98115500|
|2010-10-26 00:00:00|        308.049988| 98232400|
|2010-10-27 00:00:00|307.83000899999996| 99750700|
|2010-10-28 00:00:00|         305.23999|137762800|
|2010-10-29 00:00:00|            300.98|107627800|
|2010-11-01 00:00:00|        304.179993|105972300|
|2010-11-02 00:00:00|        309.360004|108482500|
|2010-11-03 00:00:00|        312.800003|127087100|
|2010-11-04 00:00:00|        31

In [26]:
df2.filter((df2["Close"] < 500) & ~(df2["Open"] > 300)).show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [27]:
results = df2.filter((df2["Close"] < 500) | ~(df2["Open"] > 300)).collect()

In [30]:
results[0].asDict()["Volume"]

123432400

In [31]:
df3 = spark.read.csv("sales_info.csv", inferSchema= True, header= True)

In [33]:
df3.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [38]:
df3.groupBy("Company").min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [3]:
spark.sql("CREATE TABLE employees (name STRING, age INT, salary DOUBLE)")

23/02/22 15:56:59 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/02/22 15:57:01 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/02/22 15:57:01 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/02/22 15:57:05 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/02/22 15:57:05 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore aryaman_sharan@127.0.1.1
23/02/22 15:57:05 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
23/02/22 15:57:05 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance o

DataFrame[]

In [5]:
spark.sql("CREATE TABLE temp (name STRING, age INT, salary DOUBLE)")

# spark.sql("INSERT INTO employees VALUES ('John Doe', 35, 50000.0), ('Jane Smith', 28, 60000.0), ('Bob Johnson', 42, 75000.0)")


23/02/22 16:02:37 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/02/22 16:02:37 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/02/22 16:02:37 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/02/22 16:02:37 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/02/22 16:02:37 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/02/22 16:02:37 WARN HiveMetaStore: Location: file:/mnt/c/Users/aryaman.sharan/Desktop/MLE%20Cloud%20and%20Spark/spark-warehouse/temp specified for non-external table:temp


DataFrame[]

In [7]:
results = spark.sql("INSERT INTO temp VALUES ('John Doe', 35, 50000.0), ('Jane Smith', 28, 60000.0), ('Bob Johnson', 42, 75000.0)")


In [12]:
spark.sql("select * from temp").show()

+-----------+---+-------+
|       name|age| salary|
+-----------+---+-------+
|   John Doe| 35|50000.0|
|   John Doe| 35|50000.0|
| Jane Smith| 28|60000.0|
| Jane Smith| 28|60000.0|
|Bob Johnson| 42|75000.0|
|Bob Johnson| 42|75000.0|
+-----------+---+-------+



In [13]:
spark.sql("""create table temp2 
select * from temp
where salary >= 60000""")

23/02/22 16:16:34 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/02/22 16:16:34 WARN HiveMetaStore: Location: file:/mnt/c/Users/aryaman.sharan/Desktop/MLE%20Cloud%20and%20Spark/spark-warehouse/temp2 specified for non-external table:temp2


DataFrame[]

In [15]:
results = spark.sql("select * from temp2")

In [16]:
results.show()

+-----------+---+-------+
|       name|age| salary|
+-----------+---+-------+
| Jane Smith| 28|60000.0|
| Jane Smith| 28|60000.0|
|Bob Johnson| 42|75000.0|
|Bob Johnson| 42|75000.0|
+-----------+---+-------+



In [18]:
spark.sql("select * from temp2").show()

+-----------+---+-------+
|       name|age| salary|
+-----------+---+-------+
| Jane Smith| 28|60000.0|
| Jane Smith| 28|60000.0|
|Bob Johnson| 42|75000.0|
|Bob Johnson| 42|75000.0|
+-----------+---+-------+



In [ ]:
df.re